In [ ]:
# Google Colaboratory를 사용할 때는 다음 주석을 해제하고 실행하기 바랍니다.

In [ ]:
# # 다음을 실행하면 authorization code 입력을 요청받습니다.
# # 출력된 링크를 클릭하고 Google 계정으로 로그인한 뒤,
# # authorization code를 복사해서 붙여 넣습니다.
# import os
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# working_dir = 'MLSys_100Knocks' #　※※ 여러분이 만든 폴더 경로가 다를 때는 다음을 변경합니다. ※※
# path = f'/content/drive/MyDrive/{working_dir}/MainChapter/chapter02'
# os.chdir(path)
# ! pip install japanize-matplotlib

# 2장 데이터를 시각화하고 분석하기 위한 테크닉 10



### 테크닉 11: 데이터를 로딩하고 불필요한 항목을 제외하자

In [ ]:
import pandas as pd

order_data = pd.read_csv('order_data.csv')
print(len(order_data))
order_data.head()

In [ ]:
order_data = order_data.loc[(order_data['status'] == 1) | 
                            (order_data['status'] == 2)]
print(len(order_data))
order_data.columns

In [ ]:
analyze_data = order_data[[
    'store_id', 'customer_id', 'coupon_cd',
    'order_accept_date', 'delivered_date', 'total_amount',
    'store_name', 'wide_area', 'narrow_area',
    'takeout_name', 'status_name']]
print(analyze_data.shape)
analyze_data.head()

### 테크닉 12: 데이터 전체 이미지를 파악하자

In [ ]:
analyze_data.describe()

In [ ]:
analyze_data.dtypes

In [ ]:
analyze_data[['store_id', 'coupon_cd']] = analyze_data[
    ['store_id', 'coupon_cd']].astype(str)
analyze_data.dtypes

In [ ]:
import warnings

warnings.filterwarnings('ignore')

In [ ]:
analyze_data.describe()

### 테크닉 13: 월별 매출을 집계하자

In [ ]:
analyze_data['order_accept_date'] = pd.to_datetime(
    analyze_data['order_accept_date'])
analyze_data['order_accept_month'] = analyze_data[
    'order_accept_date'].dt.strftime('%Y%m')
analyze_data[['order_accept_date', 'order_accept_month']].head()

In [ ]:
analyze_data['delivered_date'] = pd.to_datetime(
    analyze_data['delivered_date'])
analyze_data['delivered_month'] = analyze_data[
    'delivered_date'].dt.strftime('%Y%m')
analyze_data[['delivered_date', 'delivered_month']].head()

In [ ]:
analyze_data.dtypes

In [ ]:
month_data = analyze_data.groupby('order_accept_month')
month_data.describe()

In [ ]:
month_data.sum()

### 테크닉 14: 월별 추이를 시각화하자

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# 한글 폰트 처리
import os

if os.name == 'nt':  # Windows
    plt.rc('font', family='Malgun Gothic')
elif os.name == 'posix':  # macOS
    plt.rc('font', family='AllieGothic')

plt.rc('axes', unicode_minus=False)  # minus font settings

month_data.sum().plot()

In [ ]:
month_data.mean().plot()

### 테크닉 15: 매출로부터 히스토그램을 만들자

In [ ]:
plt.hist(analyze_data['total_amount'])

In [ ]:
plt.hist(analyze_data['total_amount'], bins=21)

### 테크닉 16: 시도군구별 매출을 집계해서 시각화하자

In [ ]:
pre_data = pd.pivot_table(
    analyze_data, index='order_accept_month',
    columns='narrow_area', values='total_amount',
    aggfunc='mean')
pre_data

In [ ]:
plt.plot(list(pre_data.index), pre_data['서울'], label='서울')
plt.plot(list(pre_data.index), pre_data['부산'], label='부산')
plt.plot(list(pre_data.index), pre_data['대전'], label='대전')
plt.plot(list(pre_data.index), pre_data['광주'], label='광주')
plt.plot(list(pre_data.index), pre_data['세종'], label='세종')
plt.plot(list(pre_data.index), pre_data['경기남부'], label='경기남부')
plt.plot(list(pre_data.index), pre_data['경기북부'], label='경기북부')
plt.legend()

### 테크닉 17: 클러스터링을 위해 데이터를 가공하자

In [ ]:
store_clustering = analyze_data.groupby('store_id').agg(
    ['size', 'mean', 'median', 'max', 'min'])['total_amount']
store_clustering.reset_index(inplace=True, drop=True)
print(len(store_clustering))
store_clustering.head()

In [ ]:
import seaborn as sns
hexbin = sns.jointplot(x='mean', y='size', data=store_clustering, kind='hex')

### 테크닉 18: 클러스터링을 이용해 매장을 그룹화하자

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
store_clustering_sc = sc.fit_transform(store_clustering)

kmeans = KMeans(n_clusters=4, random_state=0)
clusters = kmeans.fit(store_clustering_sc)
store_clustering['cluster'] = clusters.labels_
print(store_clustering['cluster'].unique())
store_clustering.head()

### 테크닉 19: 그룹의 경향을 분석하자

In [ ]:
store_clustering.columns = ['월 건수', '월 평균값', '월 중앙값', 
                            '월 최댓값', '월 최솟값', 'cluster']
store_clustering.groupby('cluster').count()

In [ ]:
store_clustering.groupby('cluster').mean()

### 테크닉 20: 클러스터링 결과를 t-SNE로 시각화하자

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
x = tsne.fit_transform(store_clustering_sc)
tsne_df = pd.DataFrame(x)
tsne_df['cluster'] = store_clustering['cluster']
tsne_df.columns = ['axis_0', 'axis_1', 'cluster']
tsne_df.head()

In [ ]:
tsne_graph = sns.scatterplot(x='axis_0', y='axis_1', hue='cluster', data=tsne_df)